In [42]:
import pandas as pd
file_path = 'Impact_info.xlsx'
df = pd.read_excel(file_path)

             ## 1 여기 바꾸기 ##
file_path = 'FSL_sub_14.csv'
df2 = pd.read_csv(file_path) 

C:\Users\LEE AYOON\AppData\Local\Temp\ipykernel_41460\1272389732.py:7: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(file_path)


df2(데이터프레임) str -> int/float 변환

In [43]:
# 첫 번째 열의 이름 가져오기
first_column = df2.columns[0]

# 첫 번째 열은 6번째 행부터 int로 변환
df2[first_column].iloc[5:] = df2[first_column].iloc[5:].astype(int)

# 나머지 열은 6번째 행부터 float로 변환
for column in df2.columns[1:]:
    df2[column].iloc[5:] = df2[column].iloc[5:].astype(float)

df2.head()

,Unnamed: 0,14_12_finish_ch.c3d,14_12_finish_ch.c3d.1,14_12_finish_ch.c3d.2,14_12_finish_ch.c3d.3,14_12_finish_ch.c3d.4,14_12_finish_ch.c3d.5,14_12_finish_ch.c3d.6,14_12_finish_ch.c3d.7,14_12_finish_ch.c3d.8,...,14_12_finish_ch.c3d.35,14_12_finish_ch.c3d.36,14_12_finish_ch.c3d.37,14_12_finish_ch.c3d.38,14_12_finish_ch.c3d.39,14_12_finish_ch.c3d.40,14_12_finish_ch.c3d.41,14_12_finish_ch.c3d.42,14_12_finish_ch.c3d.43,14_12_finish_ch.c3d.44
0,NaN,Head center,Head center,Head center,L ankle JC,L ankle JC,L ankle JC,L elbow JC,L elbow JC,L elbow JC,...,R knee JC,R wrist JC,R wrist JC,R wrist JC,RIGHT_HIP,RIGHT_HIP,RIGHT_HIP,Rsh_JC,Rsh_JC,Rsh_JC
1,NaN,LANDMARK,LANDMARK,LANDMARK,LANDMARK,LANDMARK,LANDMARK,LANDMARK,LANDMARK,LANDMARK,...,LANDMARK,LANDMARK,LANDMARK,LANDMARK,LANDMARK,LANDMARK,LANDMARK,LANDMARK,LANDMARK,LANDMARK
2,NaN,ORIGINAL,ORIGINAL,ORIGINAL,ORIGINAL,ORIGINAL,ORIGINAL,ORIGINAL,ORIGINAL,ORIGINAL,...,ORIGINAL,ORIGINAL,ORIGINAL,ORIGINAL,ORIGINAL,ORIGINAL,ORIGINAL,ORIGINAL,ORIGINAL,ORIGINAL
3,ITEM,X,Y,Z,X,Y,Z,X,Y,Z,...,Z,X,Y,Z,X,Y,Z,X,Y,Z
4,1,0.002773,0.008877,1.606474,-0.195343,0.003189,0.108242,-0.437862,-0.004441,1.199778,...,0.486619,0.474425,0.101624,1.434334,0.062001,0.067743,0.868732,0.1552,0,1.3656


Impact_info.xlsx 에서 파일 이름 일치하는 임팩트 타이밍 프레임 뽑아서 리스트 생성


In [44]:
# 파일 정보 열 가져오기   ### 기술동작마다 확인 필요 ###
third_column_name = df.columns[11]

# 조건에 맞는 행 필터링                                               ## 2 여기 바꾸기 / 파일이름주의 ##
filtered_rows = df[df[third_column_name].astype(str).str.startswith('0816_14_12', na=False)]

# 조건에 맞는 행에서 I 열의 데이터를 리스트에 담기 ### 기술동작마다 확인 필요 ###
data_list = []

if not filtered_rows.empty:
    for index, row in filtered_rows.iterrows():
        if len(row) >= 8:  # 행에 8개 이상의 열이 있는지 확인
            try:
                data_list.append(int(row.iloc[16]))
            except ValueError:
                data_list.append(int(float(row.iloc[16])))  # 부동소수점으로 저장된 정수 처리
else:
    print("조건에 맞는 데이터 없음.")

# 결과 리스트 출력
print("Data list:", data_list)

# df2에 47번째 열 추가 (초기값을 0으로 설정)
new_column_name = df2.columns[46] if len(df2.columns) > 46 else 'binary'
df2[new_column_name] = 0

# df2의 첫 번째 열 값을 문자열로 변환하고 공백 제거
df2[df2.columns[0]] = df2[df2.columns[0]].astype(str).str.strip()

# data_list와 df2의 첫 번째 열 비교하여 df2의 47번째 열 수정
matching_indices = []
for data in data_list:
    indices = df2[df2[df2.columns[0]] == str(data)].index
    matching_indices.extend(indices)

# 인덱스의 앞뒤로 한 행씩도 1로 라벨링
for index in matching_indices:
    df2.loc[index, new_column_name] = 1
    if index > 0:
        df2.loc[index - 1, new_column_name] = 1
    if index < len(df2) - 1:
        df2.loc[index + 1, new_column_name] = 1

# 결과 출력
print("Labeled DataFrame with indices:")
print(df2[df2[new_column_name] == 1])

           ### 3 여기도 바꿔주기 ###
df2.to_csv('FSL_sub_14_label.csv', index=False)

Data list: [1492, 1809, 2119, 2450, 2767, 3362, 3661, 3946, 4262, 5427, 4545, 4846, 5735, 6023, 6348, 6652, 6930, 7228, 7523]
Labeled DataFrame with indices:
     Unnamed: 0 14_12_finish_ch.c3d 14_12_finish_ch.c3d.1  \
1494       1491            1.523263              0.774818   
1495       1492            1.526883              0.784425   
1496       1493            1.530271               0.79373   
1811       1808            1.087594              0.691687   
1812       1809            1.088279              0.698711   
1813       1810            1.089191              0.705565   
2121       2118             0.99742              0.906703   
2122       2119            0.999653              0.914605   
2123       2120            1.001549              0.922478   
2452       2449            1.581905              0.949244   
2453       2450            1.585741              0.956951   
2454       2451              1.5896               0.96448   
2769       2766            0.698779              